In [ ]:
pip install tensorflow numpy


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:

import tensorflow as tf

model = tf.keras.models.load_model('/content/bestmodel (1).keras')


In [ ]:
import tensorflow as tf

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(5,)),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Save the model
save_path = model.save('/content/bestmodel (1).keras')

# Get the path of the saved model
print('Model saved at:', save_path)

Model saved at: None


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

image_path = '/content/drive/MyDrive/Colab Notebooks/newdataset split/test/Non-cancerous/12.png'
image_size = (224, 224)  # Adjust according to your model's input size

image = load_img(image_path, target_size=image_size)
image_array = img_to_array(image)
image_array = image_array / 255.0  # Normalize pixel values
image_array = tf.expand_dims(image_array, axis=0)  # Add batch dimension


In [ ]:
predictions = model.predict(image_array)


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueErro

In [ ]:
print(predictions)

NameError: name 'predictions' is not defined

### API

In [ ]:
!pip install -U flask-cors
!pip install flask-ngrok

In [ ]:
from flask import Flask,request, jsonify
from flask_ngrok import run_with_ngrok
from flask_cors import CORS, cross_origin
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
# from io import BytesIO
import os
model = tf.keras.models.load_model('/content/bestmodel (1).keras')


app =Flask(__name__)
CORS(app)
cors = CORS(app, resources={r"/*": {"origins": "*", "allow_headers": "*", "expose_headers": "*"}})
run_with_ngrok(app)
UPLOAD_FOLDER = '/content/Images'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/')
def index():
    return jsonify("Welcome to the home page")

@app.route('/identify_disease', methods=['POST'])
def upload_image():
    try:
      print(request.data)
      if 'image' in request.files:
          image = request.files['image']
          # image.save(image.filename)
          # print(image)
          # Save the uploaded image to the specified folder
          image_path = os.path.join(app.config['UPLOAD_FOLDER'], image.filename)
          image.save(image_path)
          print("Image saved at:", image_path)
          image_size = (224, 224)  # Adjust according to your model's input size
          image = load_img(image_path, target_size=image_size)
          image_array = img_to_array(image)
          image_array = image_array / 255.0  # Normalize pixel values
          image_array = tf.expand_dims(image_array, axis=0)  # Add batch dimension
          predictions = model.predict(image_array)
          print(predictions[0][0])
          value=predictions[0][0]
          if value>0.5:
            prediction="Cancerous"
          else:
            prediction="Non-cancerous"
          return jsonify(prediction)
      else:
        print("dsds")
        return "No image in request"
    except Exception as e:
      print(e)
      return f"Error: {str(e)}"


app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll